This homework is made by:
* Masoumeh Bakhtiariziabari (11813105)
* Marianne de Heer Kloots (11138351)
* Tharangni Sivaji (XXXXXXXX)

# Theoretical Part [15 pts]

## 1. Hypothesis Testing – The problem of multiple comparisons [5 points]
Experimentation in AI often happens like this: 
1. Modify/Build an algorithm
2. Compare the algorithm to a baseline by running a hypothesis test.
3. If not significant, go back to step A
4. If significant, start writing a paper. 

How many hypothesis tests, m, does it take to get to (with Type I error for each test = α):
* P(m<sup>th</sup> experiment gives significant result | m experiments lacking power to reject H<sub>0</sub>)?
* P(at least one significant result | m experiments lacking power to reject H<sub>0</sub>)?

<div style="background-color: lightyellow">
*answer here*
</div>


## 2. Bias and unfairness in Interleaving experiments [10 points]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?

<div style="background-color: lightyellow">
*answer here*
</div>

# Experimental Part [85 pts]
Commercial search engines use both offline and online approach in evaluating a new search algorithm: they first use an offline test collection to compare the production algorithm (P) with the new experimental algorithm (E); if *E* statistically significantly outperforms *P* with respect to the evaluation measure of their interest, the two algorithms are then compared online through an interleaving experiment.

For the purpose of this homework we will assume that the evaluation measures of interest are:
1. Binary evaluation measures
    1. Precision at rank k,
    2. Recall at rank k,
    3. Average Precision,
2. Multi-graded evaluation measures
    1. Normalized Discounted Cumulative Gain at rank k (nDCG@k),
    2. Expected Reciprocal Rank (ERR).

Further, for the purpose of this homework we will assume that the interleaving algorithms of interest are:
Team-Draft Interleaving (Joachims. "Evaluating retrieval performance using clickthrough data". Text Mining 2003.),
Probabilistic Interleaving (Hofmann, Whiteson, and de Rijke. "A probabilistic method for inferring preferences from clicks." CIKM 2011.).
 
In an interleaving experiment the ranked results of *P* and *E* (against a user query) are interleaved in a single ranked list which is presented to a user. The user then clicks on the results and the algorithm that receives most of the clicks wins the comparison. The experiment is repeated for a number of times (impressions) and the total wins for *P* and *E* are computed. 

A Sign/Binomial Test is then run to examine whether the difference in wins between the two algorithms is statistically significant (or due to chance). Alternatively one can calculate the proportion of times the *E* wins and test whether this proportion, *p*, is greater than *p<sub>0</sub>=*0.5. This is called an 1-sample 1-sided proportion test.

One of the key questions however is **whether offline evaluation and online evaluation outcomes agree with each other**. In this homework you will determine the degree of agreement between offline evaluation measures and interleaving outcomes, by the means of simulations. A similar analysis using actual online data can be found at Chapelle et al. “Large-Scale Validation and Analysis of Interleaved Search Evaluation”.

## <font color='purple'>[Based on Lecture 1]</font>
### Step 1: <font color='darkred'>Simulate Rankings of Relevance for *E* and *P* *(5 points)*</font>

In the first step you will generate pairs of rankings of relevance, for the production *P* and experimental *E*, respectively, for a hypothetical query **q**. Assume a 3-graded relevance, i.e. `{N, R, HR}`. Construct all possible *P* and *E* ranking pairs of length 5. This step should give you about.

Example:

    P: {N N N N N}
    E: {N N N N R}
    …
    P: {HR HR HR HR R}
    E: {HR HR HR HR HR}

(Note 1: If you do not have enough computational power, sample 5000 pair uniformly at random to show your work.)

In [33]:
from itertools import product
from pprint import pprint
import random

# define collections of algorithms and relevance grades
algorithms = ['P', 'E']
relevance_grades = ['N', 'HR', 'R']

# all possible ranking sequences
# list of rankings [('HR', 'HR', 'HR', 'HR', 'HR') ... ('N', 'N', 'N', 'N', 'N')]
rankings = [ranking for ranking in product(relevance_grades, repeat=5)]

# all algorithms paired with all rankings 
# (list of lists with elements e.g. ('P', ('HR', 'HR', 'HR', 'HR', 'HR')))
algorithm_rankings = [list(product(alg, rankings)) for alg in algorithms]

# all possible pairs of P and E with their rankings
all_ranking_pairs = [pair for pair in product(*algorithm_rankings)]

# all ranking pairs except equals
ranking_pairs = [pair for pair in product(*algorithm_rankings) if pair[0][1] != pair[1][1]]

# pretty print
print('number of combinations:', len(all_ranking_pairs))
print('number of non-equal combinations:', len(ranking_pairs))
pprint(random.sample(ranking_pairs, 10))

number of combinations: 59049
number of non-equal combinations: 58806
[(('P', ('HR', 'R', 'R', 'R', 'N')), ('E', ('R', 'N', 'HR', 'R', 'HR'))),
 (('P', ('N', 'N', 'N', 'R', 'HR')), ('E', ('R', 'N', 'R', 'N', 'R'))),
 (('P', ('R', 'R', 'N', 'HR', 'R')), ('E', ('R', 'N', 'N', 'N', 'R'))),
 (('P', ('N', 'N', 'HR', 'HR', 'HR')), ('E', ('HR', 'HR', 'R', 'R', 'R'))),
 (('P', ('N', 'HR', 'N', 'R', 'R')), ('E', ('HR', 'HR', 'HR', 'HR', 'N'))),
 (('P', ('N', 'R', 'HR', 'N', 'N')), ('E', ('R', 'HR', 'HR', 'R', 'N'))),
 (('P', ('R', 'HR', 'HR', 'N', 'R')), ('E', ('HR', 'N', 'R', 'HR', 'N'))),
 (('P', ('R', 'R', 'N', 'R', 'R')), ('E', ('HR', 'HR', 'R', 'N', 'N'))),
 (('P', ('N', 'R', 'R', 'HR', 'HR')), ('E', ('HR', 'R', 'N', 'HR', 'R'))),
 (('P', ('N', 'HR', 'R', 'HR', 'R')), ('E', ('R', 'N', 'R', 'R', 'R')))]


<div style="background-color: lightyellow">
<p>We find 59049 different ranking pairs. This makes sense: each ranking pair consists of 10 relevance values (5 produced by each algorithm), all of which can take on any of the 3 grades (N, R, HR). So there should be 3<sup>10</sup> = 59049 different combinations, which matches our finding.</p>

<p>We then exclude all ranking pairs which have exactly the same relevance grade sequences, which leaves us with 58806 different combinations.</p>

<p>We have printed a randomly selected sample of 10 ranking pairs as an example.</p>
</div>

### Step 2: <font color='darkred'>Implement Evaluation Measures *(10 points)*</font>
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 

(Note 2: Some of the aforementioned measures require the total number of relevant and highly relevant documents in the entire collection – pay extra attention on how to find this)

In [55]:
# ERR

import numpy as np

def numerical(relevance_sequence):
    """
    Convert a relevance grade sequence to a sequence of numerical 
    values, based on the relevance grades given.
    E.g. ['HR', 'HR', 'HR', 'R', 'N'] returns [2, 2, 2, 1, 0]
    """
    numerical_relevance_sequence = [1 if grade == 'R' \
                                    else 2 if grade == 'HR' else 0 \
                                    for grade in relevance_sequence]
    return numerical_relevance_sequence

def R_function(g, g_max):
    return (2**g - 1)/(2**g_max)

def ERR(relevance_sequence):
    """
    Compute the ERR based on Algorithm 2 in 
    https://pdfs.semanticscholar.org/7e3c/f6492128f915112ca01dcb77c766129e65cb.pdf
    """
    p = 1
    ERR = 0
    n = len(relevance_sequence)
    g_max = max(relevance_sequence)
    
    for r in range(1, n + 1):
        g = relevance_sequence[r - 1]
        R = R_function(g, g_max)
        ERR = ERR + p * (R/r)
        p = p * (1 - R)
    return ERR

ERR_list = []
for P, E in ranking_pairs:
    ERR_P = ERR(numerical(P[1]))
    ERR_E = ERR(numerical(E[1]))
    ERR_list.append((ERR_P, ERR_E))
    
print('mean of all ERR values for P & E (should be equal):\n', 
      np.mean(ERR_list, axis=0))
print('mean of a random sample of 100 pairs:\n', 
      np.mean(random.sample(ERR_list, 100), axis=0))
pprint(ERR_list[:10])

mean of all ERR values for P & E (should be equal):
 [ 0.55613292  0.55613292]
mean of a random sample of 100 pairs:
 [ 0.54628711  0.58094141]
[(0.0, 0.15),
 (0.0, 0.1),
 (0.0, 0.1875),
 (0.0, 0.225),
 (0.0, 0.2),
 (0.0, 0.125),
 (0.0, 0.175),
 (0.0, 0.175),
 (0.0, 0.25),
 (0.0, 0.2875)]


<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 3: <font color='darkred'>Calculate the 𝛥measure *(0 points)*</font>
For the three measures and all *P* and *E* ranking pairs constructed above calculate the difference: 𝛥measure = measure<sub>E</sub>-measure<sub>P</sub>. Consider only those pairs for which *E* outperforms *P*.


In [61]:
# code
difference_ERR = [ERR[1] - ERR[0] for ERR in ERR_list \
                  if ERR[1] > ERR[0]]
print(difference_ERR[:10])

[0.15, 0.1, 0.1875, 0.225, 0.2, 0.125, 0.175, 0.175, 0.25, 0.2875]


<div style="background-color: lightyellow">
explanation/analysis?
</div>

## <font color='purple'>[Based on Lecture 2]</font>
### Step 4: <font color='darkred'>Implement Interleaving *(15 points)*</font>
Implement 2 interleaving algorithms: (1) Team-Draft Interleaving OR Balanced Interleaving, AND (2), Probabilistic Interleaving. The interleaving algorithms should (a) given two rankings of relevance interleave them into a single ranking, and (b) given the users clicks on the interleaved ranking assign credit to the algorithms that produced the rankings.

(Note 4: Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, in our case the rankings consist of relevance labels. Hence in this case (a) you will assume that E and P return different documents, (b) the interleaved ranking will also be a ranking of labels.)


In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>

## <font color='purple'>[Based on Lecture 3]</font>
### Step 5: <font color='darkred'>Implement User Clicks Simulation *(15 points)*</font>
Having interleaved all the ranking pairs an online experiment could be ran. However, given that we do not have any users (and the entire homework is a big simulation) we will simulate user clicks.

We have considered a number of click models including:
1. Random Click Model (RCM)
2. Position-Based Model (PBM)
3. Simple Dependent Click Model (SDCM)
4. Simple Dynamic Bayesian Network (SDBN)

Consider two different click models, (a) the Random Click Model (RCM), and (b) one out of the remaining 3 aforementioned models. The parameters of some of these models can be estimated using the Maximum Likelihood Estimation (MLE) method, while others require using the Expectation-Maximization (EM) method. Implement the two models so that (a) there is a method that learns the parameters of the model given a set of training data, (b) there is a method that predicts the click probability given a ranked list of relevance labels, (c) there is a method that decides - stochastically - whether a document is clicked based on these probabilities.

Having implemented the two click models, estimate the model parameters using the Yandex Click Log [[file]](https://drive.google.com/file/d/1tqMptjHvAisN1CJ35oCEZ9_lb0cEJwV0/view).

(Note 6: Do not learn the attractiveness parameter *a*<sub>uq</sub>)

In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 6: <font color='darkred'>Simulate Interleaving Experiment *(10 points)*</font>
Having implemented the click models, it is time to run the simulated experiment.

For each of interleaved ranking run N simulations for each one of the click models implemented and measure the proportion *p* of wins for E.
(Note 7: Some of the models above include an attractiveness parameter *a*<sub>uq</sub>. Use the relevance label to assign this parameter by setting *a*<sub>uq</sub> for a document u in the ranked list accordingly. (See [Click Models for Web Search](http://clickmodels.weebly.com/uploads/5/2/2/5/52257029/mc2015-clickmodels.pdf))


In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 7: <font color='darkred'>Results and Analysis *(30 points)*</font>
Compare the results of the offline experiments (i.e. the values of the 𝛥measure) with the results of the online experiment (i.e. proportion of wins), analyze them and reach your conclusions regarding their agreement.
* Use easy to read and comprehend visuals to demonstrate the results;
* Analyze the results on the basis of
    * the evaluation measure used,
    * the interleaving method used,
    * the click model used.
* Report and ground your conclusions.

(Note 8: This is the place where you need to demonstrate your deeper understanding of what you have implemented so far; hence the large number of points assigned. Make sure you clearly do that so that the examiner of your work can grade it accordingly.)

<u>Yandex Click Log File</u>:

The dataset includes user sessions extracted from Yandex logs, with queries, URL rankings and clicks. To allay privacy concerns the user data is fully anonymized. So, only meaningless numeric IDs of queries, sessions, and URLs are released. The queries are grouped only by sessions and no user IDs are provided. The dataset consists of several parts. Logs represent a set of rows, where each row represents one of the possible user actions: query or click.

In the case of a Query:

    SessionID TimePassed TypeOfAction QueryID RegionID ListOfURLs


In the case of a Click:

    SessionID TimePassed TypeOfAction URLID


* `SessionID` - the unique identifier of the user session.
* `TimePassed` - the time elapsed since the beginning of the current session in standard time units.
* `TypeOfAction` - type of user action. This may be either a query (Q), or a click (C).
* `QueryID` - the unique identifier of the request.
* `RegionID` - the unique identifier of the country from which a given query. This identifier may take four values.
* `URLID` - the unique identifier of the document.
* `ListOfURLs` - the list of documents from left to right as they have been shown to users on the page extradition Yandex (top to bottom).


In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>